In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io
import itertools as it
import scipy.special as psi
plt.style.use('classic')
import seaborn as sns
import pandas as pd
import math as mt
import time
import sys

sys.path.insert(1, '/rds/general/user/lr4617/home/4th_Year_Project/Final_Year_Project/')

from scipy.io import loadmat
from scipy import stats
from numpy.random import seed
from numpy.random import rand
from scipy.integrate import quad
from scipy.io import savemat
from tempfile import TemporaryFile
from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.decomposition import KernelPCA
from mpl_toolkits import mplot3d
from mPE_fn import mPE_
from scipy.spatial import distance
from scipy.stats import entropy
from mPE_ultis import integrand, ubble, array_list, permutation
from util import rolling_mean, probability, probability_v2, get_mPE_matrix
from sklearn.cluster import KMeans

In [3]:
rats = np.load('/rds/general/user/lr4617/home/4th_Year_Project/Final_Report/control_analysis/rats_sampling_10_window_150.npy')
lengths = np.load('/rds/general/user/lr4617/home/4th_Year_Project/Final_Report/control_analysis/lengths.npy')

to_be_removed = '.ipynb_checkpoints'

root_dir = '/rds/general/user/lr4617/home/4th_Year_Project/CAPTURE_rat_multidimensional/raw_data/'
normal_trajs = os.listdir(root_dir + 'normal')
drug_trajs_1 = os.listdir(root_dir + 'caffeine')
drug_trajs_2 = os.listdir(root_dir + 'amphetamine')


if to_be_removed in normal_trajs:
    idx = normal_trajs.index(to_be_removed)
    normal_trajs.pop(idx)
    
if to_be_removed in drug_trajs_1:
    idx = drug_trajs_1drug_trajs_1.index(to_be_removed)
    drug_trajs_1.pop(idx)

if to_be_removed in drug_trajs_2:
    idx = drug_trajs_2.index(to_be_removed)
    drug_trajs_2.pop(idx)

names = normal_trajs + drug_trajs_1 + drug_trajs_2

print(rats.shape)

(6480000, 60)


In [4]:
def mPE__(*argv):    
    g=len(argv)
    
    if g==2:
        n=argv[1]
        datain=argv[0]
        lamin=np.array([1])
        epsin=np.array([1])
    if g==3:
        n=argv[1]
        datain=argv[0]
        lamin=np.array([1])
        epsin=argv[2]
    if g==4:
        n=argv[1]
        datain=argv[0]
        lamin=argv[3]
        epsin=argv[2]
    
#    Check for the data to be in the right dimension        
    if datain.shape[0] == 1 or datain.shape[1] == 1:
        if len(datain[:,0])>len(datain[0,:]):
            datain=datain.transpose()
    else:
        if len(datain[:,1])>len(datain[1,:]):
            datain=datain.transpose()
        
    scalesno=len(epsin)
    lagno=len(lamin)
    HH=np.zeros((lagno,scalesno))
    norm_HH=np.zeros((lagno,scalesno))
#    Definition of parameters: fac is the number of possible permutations
#    Elem is the No of data points
#    Dim is the dimensionality of the samples
    
    for lam in range(0,lagno):
        for eps in range(0,scalesno):
            scale=epsin[eps]
            L=lamin[lam]
            xlen=len(datain[0,:])
            ylen=np.round((xlen/scale)-0.5)
            ylen=ylen.astype(int)
            D=np.zeros((len(datain[:,0]),ylen))
            for ylenc in range(0,ylen):
                dfg=datain[:,((ylenc)*scale):((ylenc+1)*scale)]
                fhk=np.sum(dfg,axis=1)
                r=(1/scale)*fhk
                D[:,ylenc]=r
                
            data=D
            fac=mt.factorial(n)
            elem=len(data[0,:])
            dim=len(data[:,0])
            
        #    A is a n-by-factorial(n) matrix that shows all the 
        #    possible permutations of n elements
            A=permutation(n)
            
        #    counter is a factorial(n) square matrix that counts the recurrence of
        #    a dim-dimensional permutation
            lpi=fac*np.ones((dim))
            lpi=lpi.astype(int)
            nj = lpi.tolist()
            counter=np.zeros((nj))
            
        #    For each iteration i, a series of n points is sampled from the 
        #    data set and the corresponding permutation is identified. 
        #    The counter matrix keeps track of the numiber of times a certain
        #    permutation, or combination of permutations is observed. 
            for i in range(0, elem-n*L+L-1):
                coord=np.zeros((dim))
                for num in range(0,dim):
                    sample=data[num,range(i,i+L*(n),L)]
                    ord=ubble(sample)
                    perm_num=0
                    check_1=1
                    check_2=1
                    
                    while (perm_num<=fac) and (check_2):
                       
                        check_1=1
                        for j in range(0,n-1):
                            if ord[j]!=A[perm_num,j]:
                                check_1=0
                        if check_1:
                            coord[num]=perm_num
                            check_2=0
                            
                        perm_num=perm_num+1
                
                    
                coord=coord.astype(int)
                coord1=tuple(coord)
                counter[coord1]=counter[coord1]+1
                
        #    Once the counter matrix is complete, each element is divided by the
        #    total number of samples to get a empirical probability, and the PE is
        #    computed according to the method described in Schurmann
            
            H=0
            counter1=counter.flatten()
            for iter in range(0,(fac**dim)):
               
                rec=counter1[iter]
                tot=elem-n+1
                
                if rec==0:
                    H=H
                    
                else:
                    I = quad(integrand, 0, 1, args=(rec))
                    I=I[0]
                    coeff=((-1)**rec)*I
                    prob=(rec/tot)*(psi.digamma(tot)-psi.digamma(rec)-coeff)
                    H=H+mt.log2(mt.exp(prob))
                 
        #     The normalised entropy norm_H is computed as well
            norm_H=H/(mt.log2(fac**dim))
            
            HH[lam,eps]=H
            norm_HH[lam,eps]=norm_H
    return [HH,norm_HH]

In [5]:
def get_mPE_matrix_(reduced_traj, bins_number, traj_number, orders, random):
    if random:
        bins_number = 2
        mPE_vector = np.zeros((bins_number, traj_number, len(orders)))
        traj_length = int((reduced_traj.shape[0]/bins_number)/traj_number)
        print(mPE_vector.shape)

        for i in range(bins_number):
            idx = 0
            for j in range(0, traj_length*traj_number, traj_length):
                idx_1 = np.random.randint(np.max(reduced_traj.shape) - traj_length)
                traj = reduced_traj[idx_1: idx_1 + traj_length]
                [HH, _]=mPE__(traj, orders[0])
                mPE_vector[i, idx, 0] = HH
                idx +=1

    else:
        mPE_vector = np.zeros((bins_number, traj_number, len(orders)))
        traj_length = int((reduced_traj.shape[0]/bins_number)/traj_number)
        
        for a, order in enumerate(orders):
            
            for i in range(bins_number):
                idx = 0
                
                for j in range(0, traj_length*traj_number, traj_length):
                    idx_1 = i*traj_number*traj_length 
                    traj = reduced_traj[idx_1 + j: idx_1 + j + traj_length]
                        
                    if traj.shape[0]>0:
                        [HH, _]=mPE__(traj, order)
                        mPE_vector[i, idx, a] = HH
                        
                    idx +=1
    
    return mPE_vector

In [6]:
###################################################################################################################################################
####################################################################### PCA #######################################################################
###################################################################################################################################################

labels = ['HeadF', 'HeadB', 'HeadL', 'SpineF', 'SpineM',   
          'SpineL', 'Offset1', 'Offset2', 'HipL', 'HipR',     
          'ElbowL', 'ArmL', 'ShoulderL', 'ShoulderR','ElbowR',
          'ArmR', 'KneeR', 'KneeL', 'ShinL', 'ShinR']

unit_length_entropy = 3000
nPCs = [3]
max_length = 334800

use_all = False 
use_partition_up = False 
use_partition_down = False  
use_single = True 

if use_single:
    marker = 18
    nPCs = [3]
    use_PCA = False
    add_on = '_marker_'
    
    dims = np.arange(marker*3, (marker*3)+3)
    marker_name= labels[marker]

if use_partition_up:
    nPCs = [3]
    dims_1 = np.arange(24)
    dims_2 = np.arange(36,42)
    use_PCA = True
    add_on = '_partition_up'
    
    dims = np.concatenate((dims_1,dims_2), axis=0)

if use_partition_down:
    nPCs = [3]
    dims_1 = np.arange(24,36)
    dims_2 = np.arange(42,60)
    use_PCA = True
    add_on = '_partition_down'
    
    dims = np.concatenate((dims_1,dims_2), axis=0)
    
if use_all:
    dims = np.arange(60)
    use_PCA = True
    

for n_PC in nPCs:
    
    count = 0
    
    for which_traj, length in enumerate(lengths):
        
        ##### formatting for name #####
        if count <= 3:
            mode = 'normal'
        elif count >= 4 and count <=7:
            mode = 'caffeine'
        else:
            mode = 'amphetamine'
        
        name = names[count]
        print(mode, name)

        ##### retireving single trajectory #####
        if which_traj == 0:
            idx = 0
        else:
            idx += lengths[which_traj-1]

        traj = rats[idx:idx+length, dims]

        ##### apply PCA if necessary #####
        if use_PCA:
            print('########################## APPLY PCA ##########################')

            pca = PCA()
            pca.fit(traj)

            ##### reduce data according to explained variance values using linear PCA #####
            pca = PCA(n_components=n_PC)
            reduced_traj = pca.fit_transform(traj)
            if reduced_traj.shape[0] > max_length:
                reduced_traj = reduced_traj[0:max_length, :]

        else:
            if traj.shape[0] > max_length:
                reduced_traj = traj[0:max_length, :]
        
        print('signal shape: ', reduced_traj.shape)
        
        ##### Control vector assemble #####
        if count == 0 or count ==  4 or count == 8:
            control_vector = reduced_traj
        else:
            control_vector = np.concatenate((control_vector, reduced_traj),axis=0)

        ###################################################################################################################################################
        ########################################################### CONTROL ANALYSIS ######################################################################
        ###################################################################################################################################################
                
        if count == 3 or count ==  7 or count == 11:
            
            print('########################## CONTROL ANALYSIS ##########################')

            ##### params #####
            unit_length = unit_length_entropy
            bins_number = 2
            bin_length = int(control_vector.shape[0]/bins_number)
            traj_number = int(bin_length/unit_length)
            orders = [3]
            random = True

            too_low = True

            while too_low:
                
                ##### Calculate mPE #####
                mPE_vector = get_mPE_matrix_(control_vector, bins_number, traj_number, orders, random)

                ##### Calculate p-value #####
                [_, p_value] = stats.ks_2samp(mPE_vector[0, :, 0], mPE_vector[1, :, 0])

                print(p_value)

                if p_value > 0.35:
                    too_low = False
                    break
            
            path_out = '/rds/general/user/lr4617/home/4th_Year_Project/Final_Report/RESULTS/mPE_vs_time/mPE_vs_time' + str(add_on) + marker_name + '/control/' + mode + '/'
            name_out = name + '.npy'
            np.save(path_out + name_out, mPE_vector)

        ###################################################################################################################################################
        ########################################################### ENTROPY ANALYSIS ######################################################################
        ###################################################################################################################################################

        print('########################## ENTROPY ANALYSIS ##########################')

        ########## Entropy Calculation ##########
        unit_length = unit_length_entropy
        bins_number = 2
        traj_number = int(reduced_traj.shape[0]/2/unit_length)
        orders = [3]

        mPE_vector = get_mPE_matrix(reduced_traj, bins_number, traj_number, orders, random=False)
        path_out = '/rds/general/user/lr4617/home/4th_Year_Project/Final_Report/RESULTS/mPE_vs_time/mPE_vs_time' + str(add_on) + marker_name + '/result/' + mode + '/' + name + '.npy'
        np.save(path_out, mPE_vector)

        count +=1

normal traj_5
signal shape:  (334800, 3)
########################## ENTROPY ANALYSIS ##########################
normal traj_3
signal shape:  (334800, 3)
########################## ENTROPY ANALYSIS ##########################
normal traj_2
signal shape:  (334800, 3)
########################## ENTROPY ANALYSIS ##########################
normal traj_1
signal shape:  (334800, 3)
########################## CONTROL ANALYSIS ##########################
(2, 223, 1)
0.15116060173302362
(2, 223, 1)
0.695274225375262
########################## ENTROPY ANALYSIS ##########################
caffeine traj_8
signal shape:  (334800, 3)
########################## ENTROPY ANALYSIS ##########################
caffeine traj_4
signal shape:  (334800, 3)
########################## ENTROPY ANALYSIS ##########################
caffeine traj_2
signal shape:  (334800, 3)
########################## ENTROPY ANALYSIS ##########################
caffeine traj_6
signal shape:  (334800, 3)
########################## CONTROL